In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

In [2]:
# Step 1: Generate Random Data (Simulating Image Data)
def generate_dummy_data(num_samples=1000, img_size=(32, 32, 3)):
    images = np.random.rand(num_samples, *img_size).astype(np.float32)  # Random images
    labels = np.random.choice([0, 1, 2, 3], num_samples)  # Random rotation labels (0°, 90°, 180°, 270°)
    return images, labels

# Step 2: Preprocessing
def preprocess_data(images, labels):
    images = images / 255.0  # Normalize pixel values
    images = np.transpose(images, (0, 3, 1, 2))  # Convert to PyTorch format (C, H, W)
    labels = torch.tensor(labels, dtype=torch.long)
    images = torch.tensor(images, dtype=torch.float32)
    return images, labels

# Step 3: Train-Test Split
images, labels = generate_dummy_data()
images, labels = preprocess_data(images, labels)
dataset = TensorDataset(images, labels)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [3]:
# Step 4: Define Base Model (CNN for Rotation Prediction)
class RotNet(nn.Module):
    def __init__(self):
        super(RotNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 4)  # 4 classes for rotations

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
# Step 5: Train Base Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RotNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Accuracy: {correct/total:.2%}")

train_model(model, train_loader, criterion, optimizer)

Epoch 1, Loss: 1.3866, Accuracy: 25.12%
Epoch 2, Loss: 1.3846, Accuracy: 27.12%
Epoch 3, Loss: 1.3842, Accuracy: 27.12%
Epoch 4, Loss: 1.3844, Accuracy: 27.12%
Epoch 5, Loss: 1.3844, Accuracy: 27.12%


In [5]:
# Step 6: Evaluate Model
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.2%}")

evaluate_model(model, test_loader)

Test Accuracy: 26.50%


In [7]:
# Step 7: Predict New Data
def predict_new_data(model, num_samples=5, img_size=(32, 32, 3)):
    model.eval()
    new_images = np.random.rand(num_samples, *img_size).astype(np.float32) / 255.0  # Random new images
    new_images = np.transpose(new_images, (0, 3, 1, 2))  # Convert to PyTorch format
    new_images = torch.tensor(new_images, dtype=torch.float32).contiguous().to(device)  # Ensure contiguous memory
    with torch.no_grad():
        outputs = model(new_images)
        predicted_labels = torch.argmax(outputs, dim=1).cpu().numpy()
    print("Predicted Rotation Labels:", predicted_labels)
    return predicted_labels

predict_new_data(model)

Predicted Rotation Labels: [2 2 2 2 2]


array([2, 2, 2, 2, 2])